In [2]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']


#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
#     for i in range(3):
#         temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
#     for i in range(3):
#         temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
#     temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
#     temp1 = temp.reset_index()
#     temp2 = temp.groupby('user_age_level').sum().reset_index()
#     temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
#     temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#     temp3['age_pv'] = temp4
#     temp3.drop('instance_id_x', axis=1, inplace=True)
#     temp3.drop('instance_id_y', axis=1, inplace=True)
#     data = pd.merge(data,temp3,on=['user_age_level','user_age_level'],how='left')
#     del(temp3)
#     del(temp2)
#     del(temp1)
#     del(temp4)
#     del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data



In [3]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False) 
train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [4]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 50)
2018-09-24 23:59:47
(357066, 50)
(121021, 50)


In [5]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# 0.83108
# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test
(18371, 45)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [6]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [7]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 48)


In [8]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_category_list_num',
       'item_property_list_num', 'hour_num', 'item_click',
       'shop_score_service_num', 'shop_score_description_num',
       'shop_review_positive_rate_num', 'shop_score_delivery_num',
       'user_click', 'property_0_click', 'property_1_click',
       'property_2_click', 'shop_click', 'brand_click', 'category_1_pvnum',
       'property_0_pvnum', 'property_1_pvnum', 'property_2_pvnum',
       'item_id_pvnum', 'category_0', 'category_1', 'category_2', 'property_0',
       'property_1', 'property_2', 'halfhour', 'user_count', 'item_count'],

In [9]:
# LogisticRegression?

In [10]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [11]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [12]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 10
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683897
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674839
[3]	valid_0's binary_logloss: 0.665948
[4]	valid_0's binary_logloss: 0.657237
[5]	valid_0's binary_logloss: 0.648687
[6]	valid_0's binary_logloss: 0.6403
[7]	valid_0's binary_logloss: 0.632073
[8]	valid_0's binary_logloss: 0.624002
[9]	valid_0's binary_logloss: 0.61608
[10]	valid_0's binary_logloss: 0.608305
[11]	valid_0's binary_logloss: 0.600673
[12]	valid_0's binary_logloss: 0.593178
[13]	valid_0's binary_logloss: 0.585819
[14]	valid_0's binary_logloss: 0.578595
[15]	valid_0's binary_logloss: 0.571497
[16]	valid_0's binary_logloss: 0.564521
[17]	valid_0's binary_logloss: 0.557667
[18]	valid_0's binary_logloss: 0.550932
[19]	valid_0's binary_logloss: 0.54432
[20]	valid_0's binary_logloss: 0.537819
[21]	valid_0's binary_logloss: 0.531439
[22]	valid_0's binary_logloss: 0.525149
[23]	valid_0's binary_logloss: 0.518983
[24]	valid_0's binary_logloss:

[203]	valid_0's binary_logloss: 0.12665
[204]	valid_0's binary_logloss: 0.126096
[205]	valid_0's binary_logloss: 0.125546
[206]	valid_0's binary_logloss: 0.125002
[207]	valid_0's binary_logloss: 0.124466
[208]	valid_0's binary_logloss: 0.123941
[209]	valid_0's binary_logloss: 0.12342
[210]	valid_0's binary_logloss: 0.122904
[211]	valid_0's binary_logloss: 0.122394
[212]	valid_0's binary_logloss: 0.12189
[213]	valid_0's binary_logloss: 0.121393
[214]	valid_0's binary_logloss: 0.120901
[215]	valid_0's binary_logloss: 0.120415
[216]	valid_0's binary_logloss: 0.119934
[217]	valid_0's binary_logloss: 0.119462
[218]	valid_0's binary_logloss: 0.118996
[219]	valid_0's binary_logloss: 0.118536
[220]	valid_0's binary_logloss: 0.11808
[221]	valid_0's binary_logloss: 0.11763
[222]	valid_0's binary_logloss: 0.117186
[223]	valid_0's binary_logloss: 0.116745
[224]	valid_0's binary_logloss: 0.116311
[225]	valid_0's binary_logloss: 0.115886
[226]	valid_0's binary_logloss: 0.115461
[227]	valid_0's binar

[403]	valid_0's binary_logloss: 0.0855795
[404]	valid_0's binary_logloss: 0.0855343
[405]	valid_0's binary_logloss: 0.085484
[406]	valid_0's binary_logloss: 0.08544
[407]	valid_0's binary_logloss: 0.0853939
[408]	valid_0's binary_logloss: 0.0853451
[409]	valid_0's binary_logloss: 0.0853039
[410]	valid_0's binary_logloss: 0.0852602
[411]	valid_0's binary_logloss: 0.0852145
[412]	valid_0's binary_logloss: 0.0851717
[413]	valid_0's binary_logloss: 0.0851297
[414]	valid_0's binary_logloss: 0.0850862
[415]	valid_0's binary_logloss: 0.0850453
[416]	valid_0's binary_logloss: 0.0850059
[417]	valid_0's binary_logloss: 0.0849635
[418]	valid_0's binary_logloss: 0.0849224
[419]	valid_0's binary_logloss: 0.0848826
[420]	valid_0's binary_logloss: 0.084844
[421]	valid_0's binary_logloss: 0.0848059
[422]	valid_0's binary_logloss: 0.0847693
[423]	valid_0's binary_logloss: 0.0847328
[424]	valid_0's binary_logloss: 0.0846946
[425]	valid_0's binary_logloss: 0.0846568
[426]	valid_0's binary_logloss: 0.0846

[601]	valid_0's binary_logloss: 0.0815362
[602]	valid_0's binary_logloss: 0.0815287
[603]	valid_0's binary_logloss: 0.0815222
[604]	valid_0's binary_logloss: 0.0815133
[605]	valid_0's binary_logloss: 0.0815058
[606]	valid_0's binary_logloss: 0.0815015
[607]	valid_0's binary_logloss: 0.0814906
[608]	valid_0's binary_logloss: 0.0814816
[609]	valid_0's binary_logloss: 0.0814707
[610]	valid_0's binary_logloss: 0.0814614
[611]	valid_0's binary_logloss: 0.0814534
[612]	valid_0's binary_logloss: 0.0814457
[613]	valid_0's binary_logloss: 0.0814362
[614]	valid_0's binary_logloss: 0.0814288
[615]	valid_0's binary_logloss: 0.0814231
[616]	valid_0's binary_logloss: 0.0814168
[617]	valid_0's binary_logloss: 0.081411
[618]	valid_0's binary_logloss: 0.0814019
[619]	valid_0's binary_logloss: 0.0813949
[620]	valid_0's binary_logloss: 0.0813891
[621]	valid_0's binary_logloss: 0.0813852
[622]	valid_0's binary_logloss: 0.0813777
[623]	valid_0's binary_logloss: 0.0813711
[624]	valid_0's binary_logloss: 0.0

[798]	valid_0's binary_logloss: 0.0807123
[799]	valid_0's binary_logloss: 0.0807096
[800]	valid_0's binary_logloss: 0.0807058
[801]	valid_0's binary_logloss: 0.0807035
[802]	valid_0's binary_logloss: 0.0807019
[803]	valid_0's binary_logloss: 0.0806998
[804]	valid_0's binary_logloss: 0.080698
[805]	valid_0's binary_logloss: 0.0806956
[806]	valid_0's binary_logloss: 0.0806938
[807]	valid_0's binary_logloss: 0.0806924
[808]	valid_0's binary_logloss: 0.0806901
[809]	valid_0's binary_logloss: 0.080687
[810]	valid_0's binary_logloss: 0.0806846
[811]	valid_0's binary_logloss: 0.080683
[812]	valid_0's binary_logloss: 0.0806796
[813]	valid_0's binary_logloss: 0.0806779
[814]	valid_0's binary_logloss: 0.080676
[815]	valid_0's binary_logloss: 0.0806724
[816]	valid_0's binary_logloss: 0.0806694
[817]	valid_0's binary_logloss: 0.0806657
[818]	valid_0's binary_logloss: 0.0806643
[819]	valid_0's binary_logloss: 0.0806622
[820]	valid_0's binary_logloss: 0.0806607
[821]	valid_0's binary_logloss: 0.0806

...

[1185]	valid_0's binary_logloss: 0.0802923
[1186]	valid_0's binary_logloss: 0.0802919
[1187]	valid_0's binary_logloss: 0.0802916
[1188]	valid_0's binary_logloss: 0.0802912
[1189]	valid_0's binary_logloss: 0.0802915
[1190]	valid_0's binary_logloss: 0.0802904
[1191]	valid_0's binary_logloss: 0.0802893
[1192]	valid_0's binary_logloss: 0.0802888
[1193]	valid_0's binary_logloss: 0.0802894
[1194]	valid_0's binary_logloss: 0.0802895
[1195]	valid_0's binary_logloss: 0.0802886
[1196]	valid_0's binary_logloss: 0.0802878
[1197]	valid_0's binary_logloss: 0.0802869
[1198]	valid_0's binary_logloss: 0.0802846
[1199]	valid_0's binary_logloss: 0.0802849
[1200]	valid_0's binary_logloss: 0.080284
[1201]	valid_0's binary_logloss: 0.0802841
[1202]	valid_0's binary_logloss: 0.080283
[1203]	valid_0's binary_logloss: 0.0802813
[1204]	valid_0's binary_logloss: 0.0802821
[1205]	valid_0's binary_logloss: 0.080281
[1206]	valid_0's binary_logloss: 0.0802806
[1207]	valid_0's binary_logloss: 0.0802806
[1208]	valid_0

[1377]	valid_0's binary_logloss: 0.0802351
[1378]	valid_0's binary_logloss: 0.0802344
[1379]	valid_0's binary_logloss: 0.0802329
[1380]	valid_0's binary_logloss: 0.0802308
[1381]	valid_0's binary_logloss: 0.0802299
[1382]	valid_0's binary_logloss: 0.0802294
[1383]	valid_0's binary_logloss: 0.0802302
[1384]	valid_0's binary_logloss: 0.0802299
[1385]	valid_0's binary_logloss: 0.0802308
[1386]	valid_0's binary_logloss: 0.080231
[1387]	valid_0's binary_logloss: 0.0802314
[1388]	valid_0's binary_logloss: 0.0802306
[1389]	valid_0's binary_logloss: 0.0802322
[1390]	valid_0's binary_logloss: 0.0802309
[1391]	valid_0's binary_logloss: 0.0802299
[1392]	valid_0's binary_logloss: 0.0802303
[1393]	valid_0's binary_logloss: 0.080229
[1394]	valid_0's binary_logloss: 0.0802285
[1395]	valid_0's binary_logloss: 0.0802266
[1396]	valid_0's binary_logloss: 0.0802252
[1397]	valid_0's binary_logloss: 0.0802257
[1398]	valid_0's binary_logloss: 0.0802246
[1399]	valid_0's binary_logloss: 0.0802251
[1400]	valid_

[1569]	valid_0's binary_logloss: 0.0802205
[1570]	valid_0's binary_logloss: 0.0802223
[1571]	valid_0's binary_logloss: 0.0802223
[1572]	valid_0's binary_logloss: 0.0802226
[1573]	valid_0's binary_logloss: 0.0802228
[1574]	valid_0's binary_logloss: 0.080222
[1575]	valid_0's binary_logloss: 0.0802217
[1576]	valid_0's binary_logloss: 0.0802216
[1577]	valid_0's binary_logloss: 0.0802209
[1578]	valid_0's binary_logloss: 0.0802208
[1579]	valid_0's binary_logloss: 0.0802209
[1580]	valid_0's binary_logloss: 0.0802208
[1581]	valid_0's binary_logloss: 0.08022
[1582]	valid_0's binary_logloss: 0.0802207
[1583]	valid_0's binary_logloss: 0.0802208
[1584]	valid_0's binary_logloss: 0.0802207
[1585]	valid_0's binary_logloss: 0.0802217
[1586]	valid_0's binary_logloss: 0.0802211
[1587]	valid_0's binary_logloss: 0.0802215
[1588]	valid_0's binary_logloss: 0.080222
[1589]	valid_0's binary_logloss: 0.0802209
[1590]	valid_0's binary_logloss: 0.0802217
[1591]	valid_0's binary_logloss: 0.080222
[1592]	valid_0's

[1761]	valid_0's binary_logloss: 0.0802188
[1762]	valid_0's binary_logloss: 0.0802185
[1763]	valid_0's binary_logloss: 0.0802187
[1764]	valid_0's binary_logloss: 0.0802175
[1765]	valid_0's binary_logloss: 0.0802175
[1766]	valid_0's binary_logloss: 0.0802179
[1767]	valid_0's binary_logloss: 0.080218
[1768]	valid_0's binary_logloss: 0.080217
[1769]	valid_0's binary_logloss: 0.0802178
[1770]	valid_0's binary_logloss: 0.0802179
[1771]	valid_0's binary_logloss: 0.0802174
[1772]	valid_0's binary_logloss: 0.0802173
[1773]	valid_0's binary_logloss: 0.080217
[1774]	valid_0's binary_logloss: 0.080217
[1775]	valid_0's binary_logloss: 0.0802168
[1776]	valid_0's binary_logloss: 0.0802169
[1777]	valid_0's binary_logloss: 0.0802161
[1778]	valid_0's binary_logloss: 0.080215
[1779]	valid_0's binary_logloss: 0.0802149
[1780]	valid_0's binary_logloss: 0.0802143
[1781]	valid_0's binary_logloss: 0.0802147
[1782]	valid_0's binary_logloss: 0.0802138
[1783]	valid_0's binary_logloss: 0.0802138
[1784]	valid_0's

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=10, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [13]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0808071854174
0.0802144355371


In [14]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [16]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180406.txt',sep=" ",index=False)